In [1]:
import sqlite3
import requests
import json
import warnings
import re

In [2]:
def get_marketdata(isin, try_counter = 1):
    # Подключение к API мосбиржи

    # Проверка количества попыток для подключения (максимум 4)
    if try_counter >= 4:
        warnings.warn("Попытки подключения к API мосбиржи оказались неудачными", RuntimeWarning)
        return
    
    url = f"https://iss.moex.com/iss/engines/stock/markets/bonds/securities/{isin}.json"
    response = requests.get(url)  # запрос данных по url

    # Проверка успешного подключения
    if response.status_code != 200:
        warnings.warn("Не удалось подключиться к API мосбиржи", RuntimeWarning)

        # Выполняем повторное подключение
        get_marketdata(isin, try_counter = try_counter + 1)
        return
    
    data = response.json()  # Преобразование ответа в JSON

    # Получение данных из блока securities
    inf = get_securities_block(isin, data)

    # Получение данных из блока marketdata
    inf1 = get_marketdata_block(inf, isin, data)

    # Получение данных из блока marketdata_yields
    inf2 = get_marketdata_yields_block(inf1, isin, data)

    if inf2:
        # Сохранение в базу данных
        db = DatabaseManager('bonds.db')
        db.insert_dict("bonds_info", inf2)
    else:
        print(f"Информация по {isin} не найдена")


In [5]:
def get_securities_block(isin, data) -> dict:
    # Получение данных из блока securities

    try:
        # Возможно имеет смысл добавить обработку try except для каждого получаемого поля
        
        inf = {}
        # SECID (ISIN)
        inf[data["securities"]["columns"][0]] = data["securities"]["data"][0][0]
        inf[data["securities"]["columns"][0]] = data["securities"]["data"][0][0]
        
        # get boardid (режим торгов)
        inf[data["securities"]["columns"][1]] = data["securities"]["data"][0][1]
    
        # значение купона
        inf[data["securities"]["columns"][5]] = data["securities"]["data"][0][5]
    
        # дата следующего купона
        inf[data["securities"]["columns"][6]] = data["securities"]["data"][0][6]
    
        # Lotsize (лотность)
        inf[data["securities"]["columns"][9]] = data["securities"]["data"][0][9]
    
        # facevalue (номинал)
        inf[data["securities"]["columns"][10]] = data["securities"]["data"][0][10]
    
        # status
        inf[data["securities"]["columns"][12]] = data["securities"]["data"][0][12]
    
        # matdate (Дата погашения)
        inf[data["securities"]["columns"][13]] = data["securities"]["data"][0][13]
    
        # COUPONPERIOD 
        inf[data["securities"]["columns"][15]] = data["securities"]["data"][0][15]
    
        # ISSUESIZE
        inf[data["securities"]["columns"][16]] = data["securities"]["data"][0][16]
    
        # SECNAME
        inf[data["securities"]["columns"][19]] = data["securities"]["data"][0][19]
    
        # FACEUNIT (валюта)
        inf[data["securities"]["columns"][25]] = data["securities"]["data"][0][25]
    
        # ISIN
        inf[data["securities"]["columns"][28]] = data["securities"]["data"][0][28]
    
        # COUPONPERCENT (купон в процентах)
        inf[data["securities"]["columns"][35]] = data["securities"]["data"][0][35]
    
        # OFFERDATE (дата оферты)
        inf[data["securities"]["columns"][36]] = data["securities"]["data"][0][36]

    except:
        print(f"Ошибка с ISIN {isin} в блоке securities")
        warnings.warn(f"Информация в блоке securities по бумаге {isin} не найдена", UserWarning)

    finally:
        return inf

In [7]:
def get_marketdata_block(inf, isin, data) -> dict:
    # Получение данных из блока marketdata

    try:
        # LAST (последняя цена)
        inf[data["marketdata"]["columns"][11]] = data["marketdata"]["data"][0][11]
    
        # VALUE (должна быть цена в рублях)
        inf[data["marketdata"]["columns"][15]] = data["marketdata"]["data"][0][15]
    
        # YIELD (YTM)
        inf[data["marketdata"]["columns"][16]] = data["marketdata"]["data"][0][16]
    
        # VALUE_USD (цена в долларах)
        inf[data["marketdata"]["columns"][17]] = data["marketdata"]["data"][0][17]
    
        # DURATION (в днях)
        inf[data["marketdata"]["columns"][36]] = data["marketdata"]["data"][0][36]
    
        # YIELDTOOFFER (доходность к оферте)
        inf[data["marketdata"]["columns"][56]] = data["marketdata"]["data"][0][56]

    except:
        print(f"Ошибка с ISIN {isin} в блоке marketdata")
        warnings.warn(f"Информация в блоке marketdata по бумаге {isin} не найдена", UserWarning)
        
    finally:
        return inf

In [9]:
def get_marketdata_yields_block(inf, isin, data) -> dict:
    # Получение данных из блока marketdata_yields

    try:
        # YIELDDATE (дата на которую рассчитывается доходность)
        inf[data["marketdata_yields"]["columns"][3]] = data["marketdata_yields"]["data"][0][3]
    
        # YIELDDATETYPE (тип события которое будет в дату на которую рассчитывается доходность)
        inf[data["marketdata_yields"]["columns"][5]] = data["marketdata_yields"]["data"][0][5]
    
        # EFFECTIVEYIELD (эффективная доходность)
        inf[data["marketdata_yields"]["columns"][6]] = data["marketdata_yields"]["data"][0][6]
    
        # ZSPREADBP (z спред)
        inf[data["marketdata_yields"]["columns"][8]] = data["marketdata_yields"]["data"][0][8]
    
        # GSPREADBP (g спред)
        inf[data["marketdata_yields"]["columns"][9]] = data["marketdata_yields"]["data"][0][9]
        
    except:
        print(f"Ошибка с ISIN {isin} в блоке marketdata_yields")
        warnings.warn(f"Информация в блоке marketdata_yields по бумаге {isin} не найдена", UserWarning)

    finally:
        return inf

In [68]:
class DatabaseManager:
    def __init__(self, db_path):
        self.db_path = db_path
    
    def __enter__(self):
        self.conn = sqlite3.connect(self.db_path)
        return self.conn.cursor()
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.conn.commit()
        else:
            self.conn.rollback()
        self.conn.close()

    def is_date_string(self, value):
        """
        Проверяет, является ли строка датой в формате 'YYYY-MM-DD'
        """
        if not isinstance(value, str):
            return False
        
        # Проверка формата YYYY-MM-DD
        pattern = r'^\d{4}-\d{2}-\d{2}$'
        if re.match(pattern, value):
            return True
        
        return False
    
    def insert_dict(self, table_name, data_dict):
        with self as cursor:
            # Проверяем существование таблицы
            cursor.execute("""
                SELECT name FROM sqlite_master 
                WHERE type='table' AND name=?
            """, (table_name,))
            
            table_exists = cursor.fetchone() is not None
            if not table_exists:
                # Создаем таблицу на основе ключей словаря
                columns = []
                for key, value in data_dict.items():
                    if isinstance(value, int):
                        col_type = 'INTEGER'
                    elif isinstance(value, float):
                        col_type = 'REAL'
                    elif self.is_date_string(value):
                        col_type = 'TEXT'
                    else:
                        col_type = 'TEXT'
                    columns.append(f'{key} {col_type}')
                
                create_query = f'''
                    CREATE TABLE {table_name} (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        {", ".join(columns)}
                    )
                '''
                cursor.execute(create_query)
        
            columns = ', '.join(data_dict.keys())
            placeholders = ':' + ', :'.join(data_dict.keys())
            
            # Выполняем запрос - SQLite сам преобразует None в NULL
            cursor.execute(f'''
                INSERT INTO {table_name} ({columns})
                VALUES ({placeholders})
            ''', data_dict)

    def delete_table(self, table_name):
        "Удаление таблицы"
        with self as cursor:
            
            # Проверяем существование таблицы
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
            if cursor.fetchone():
                # Удаляем таблицу
                cursor.execute(f"DROP TABLE {table_name}")

    def fetch_data_from_sqlite(self, df, keys_list, table_name):
        """Получение данных об облигациях из SQLite по списку ISIN"""

        # Проверка что список ISIN не пустой
        if not keys_list:
            print("Передан пустой список ключей")
            return

        with self as cursor:
            placeholders = ','.join(['?'] * len(keys_list))

            query = f"""
            SELECT *
            FROM {table_name} 
            WHERE isin IN ({placeholders})
            """

            # Если self возвращает курсор, получаем соединение из него
            if hasattr(cursor, 'connection'):
                conn = cursor.connection
            else:
                # Или предполагаем, что cursor это соединение
                conn = cursor

            result_df = pl.read_database(query, conn, execute_options={'parameters': keys_list})
            
            result_df = df.join(result_df, on="ISIN", how="inner")

            return result_df 

In [58]:
ISIN = "RU000A105BY1"
get_marketdata(isin=ISIN)

In [14]:
def get_currency(try_counter=1):

    # Проверка количества попыток для подключения (максимум 4)
    if try_counter >= 4:
        warnings.warn("Попытки подключения к API мосбиржи оказались неудачными", RuntimeWarning)
        return
    
    url = "https://iss.moex.com/iss/engines/currency/markets/index/securities.json"
    response = requests.get(url)  # запрос данных по url

    # Проверка успешного подключения
    if response.status_code != 200:
        warnings.warn("Не удалось подключиться к API мосбиржи", RuntimeWarning)

        # Выполняем повторное подключение
        get_marketdata(isin, try_counter = try_counter + 1)
        return
    
    data = response.json()  # Преобразование ответа в JSON

    # количество возможных для парсинга валют
    str_number = min(len(data['securities']['data']), len(data['marketdata']['data']))

    inf = {}

    # проходим по всем валютам
    for i in range(str_number):
        
        try:
            # BOARDID
            inf[data['securities']['columns'][0]] = data['securities']['data'][i][0]
        
            # SECID
            inf[data['securities']['columns'][1]] = data['securities']['data'][i][1]
        
            # SHORTNAME
            inf[data['securities']['columns'][2]] = data['securities']['data'][i][2]
        
            # LATNAME
            inf[data['securities']['columns'][3]] = data['securities']['data'][i][3]
        
            # NAME
            inf[data['securities']['columns'][4]] = data['securities']['data'][i][4]
        
            # TRADEDATE
            inf[data['marketdata']['columns'][2]] = data['marketdata']['data'][i][2]
        
            # TIME
            inf[data['marketdata']['columns'][3]] = data['marketdata']['data'][i][3]
        
            # LASTVALUE
            inf[data['marketdata']['columns'][4]] = data['marketdata']['data'][i][4]
        except:
            print(f"Ошибка при загрузке валют")
            warnings.warn(f"Информация не найдена", UserWarning)
            return inf
    
        # Сохранение в базу данных
        db = DatabaseManager('bonds.db')
        db.insert_dict("currency", inf)
        
    print("Данные о валютах обновлены")

In [55]:
get_currency()

{'BOARDID': 'FIXI', 'SECID': 'USDKZTFIXME', 'SHORTNAME': 'USDKZTFIXME', 'LATNAME': 'MOEX USD/KZT FX FIXING', 'NAME': 'Фиксинг доллар США/казахстанский тенге Московской Биржи', 'TRADEDATE': '2025-10-14', 'TIME': '12:30:00', 'LASTVALUE': 539.0608}


In [16]:
import polars as pl

In [70]:
def portfolio_upload(path="bonds.xlsx"):
    # Загрузка портфеля облигаций из эксель файла. Файл содержит 2 столбца: ISIN'ы и доля каждого isin
    try:
        df = pl.read_excel(path)
    except IOError as e:
        print(f"Не найден файл по пути {path}")
        raise e

    try:
        df.cast({'Доля' : pl.Float32})
    except:
        print("В столбце 'Доля' должны быть только числовые значения")
        raise ValueError

    if len(df.columns) != 2:
        print("Файл эксель должен состоять из 2 столбцов: 'ISIN' (строковое) и 'Доля' (float) каждого из ISIN в портфеле")
        return

    # Подключение к базе данных
    db = DatabaseManager('bonds.db')
    
    # Удаление старой таблицы с данными по облигациям
    db.delete_table("bonds_info")
    
    # обновление данных по каждому ISIN в базе данных
    for row in df.iter_rows():
        get_marketdata(row[0])

    # Уникальные ISIN из датафрейма
    unique_isins = df["ISIN"].unique().to_list()

    bond_data_df = db.fetch_data_from_sqlite(df, unique_isins, "bonds_info")
    
    return bond_data_df

In [72]:
portfolio_upload()

ISIN,Доля,id,SECID,BOARDID,COUPONVALUE,NEXTCOUPON,LOTSIZE,FACEVALUE,STATUS,MATDATE,COUPONPERIOD,ISSUESIZE,SECNAME,FACEUNIT,COUPONPERCENT,OFFERDATE,LAST,VALUE,YIELD,VALUE_USD,DURATION,YIELDTOOFFER,YIELDDATE,YIELDDATETYPE,EFFECTIVEYIELD,ZSPREADBP,GSPREADBP
str,f64,i64,str,str,f64,str,i64,i64,str,str,i64,i64,str,str,f64,null,f64,f64,f64,f64,i64,null,str,str,f64,i64,i64
"""RU000A10B4J5""",0.1,1,"""RU000A10B4J5""","""TQCB""",1.13,"""2025-10-17""",1,100,"""A""","""2027-03-11""",30,580000,"""Полипласт АО П02-БО-03""","""USD""",13.7,null,101.1076,16168.9,13.67,202.22,463,null,"""2027-03-11""","""MATDATE""",13.6707,-132,-130
"""RU000A10BS68""",0.2,2,"""RU000A10BS68""","""TQCB""",1.05,"""2025-11-08""",1,100,"""A""","""2027-11-28""",30,200000,"""Уральская Сталь БО-001Р-04""","""USD""",12.75,null,100.5,8035.87,13.25,100.5,680,null,"""2027-11-28""","""MATDATE""",13.2507,-191,-189
"""RU000A10BQV8""",0.3,3,"""RU000A10BQV8""","""TQCB""",0.99,"""2025-11-02""",1,100,"""A""","""2028-06-19""",30,620000,"""Инвест КЦ 001P-01""","""USD""",12.0,null,102.4151,40944.99,11.57,512.08,837,null,"""2028-06-19""","""MATDATE""",11.571,-365,-364
"""RU000A10CM89""",0.15,4,"""RU000A10CM89""","""TQCB""",1.19,"""2025-11-01""",1,100,"""A""","""2027-08-23""",30,500000,"""КИФА БО-01""","""CNY""",14.5,null,100.0,27797.5,15.48,347.65,594,null,"""2027-08-23""","""MATDATE""",15.4756,36,38
"""RU000A10CCJ1""",0.07,5,"""RU000A10CCJ1""","""TQCB""",1.1,"""2025-11-13""",1,100,"""A""","""2028-09-01""",31,100000,"""Славянск ЭКО 001Р-04""","""USD""",13.0,null,98.9978,7915.76,14.22,99.0,881,null,"""2028-09-01""","""MATDATE""",14.2202,-104,-102
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RU000A105BY1""",0.02,13,"""RU000A105BY1""","""TQCB""",18.5,"""2025-11-17""",1,1000,"""A""","""2028-11-17""",365,343388,"""Газпром капитал ООО ЗО28-1-Е""","""EUR""",1.85,null,84.0,544967.81,7.88,6815.6,1083,null,"""2028-11-17""","""MATDATE""",7.8799,-740,-740
"""RU000A108TV3""",0.0,14,"""RU000A108TV3""","""TQCB""",15.5,"""2025-12-25""",1,1000,"""A""","""2027-06-25""",183,198416,"""АЛРОСА З027-Д""","""USD""",3.1,null,92.69,1.3785e7,7.84,172403.4,600,null,"""2027-06-25""","""MATDATE""",7.8359,-728,-727
"""RU000A10B0C8""",0.0,15,"""RU000A10B0C8""","""TQCB""",8.38,"""2025-10-31""",1,1000,"""A""","""2027-02-23""",30,1500000,"""МЕТАЛЛОИНВЕСТ 001P-12""","""CNY""",10.2,null,103.2498,11480.35,7.94,143.58,463,null,"""2027-02-23""","""MATDATE""",7.94,-705,-703
